In [19]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [21]:
%cd /content/drive/MyDrive/5_6G_slicing_project


/content/drive/MyDrive/5_6G_slicing_project


In [22]:
import os
print(os.getcwd())
print(os.listdir("."))


/content/drive/MyDrive/5_6G_slicing_project
['data']


In [23]:
import os, tarfile

# S'assurer que les dossiers existent dans Drive
os.makedirs("data/ip_sample", exist_ok=True)
os.makedirs("data/times", exist_ok=True)

# Décompresser les archives DANS Drive
with tarfile.open("/content/ip_addresses_sample.tar.gz") as tar:
    tar.extractall("data/ip_sample")

with tarfile.open("/content/times.tar.gz") as tar:
    tar.extractall("data/times")


/tmp/ipython-input-2733344346.py:9: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall("data/ip_sample")
/tmp/ipython-input-2733344346.py:12: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall("data/times")


In [24]:
import glob, os
print(len(glob.glob("data/ip_sample/ip_addresses_sample/agg_10_minutes/*.csv")))
print(os.path.exists("data/times/times/times_10_minutes.csv"))


1000
True


In [2]:
from google.colab import files

print("Upload : ip_addresses_sample.tar.gz, times.tar.gz, weekends_and_holidays.csv")
uploaded = files.upload()


Upload : ip_addresses_sample.tar.gz, times.tar.gz, weekends_and_holidays.csv


Saving ip_addresses_sample.tar.gz to ip_addresses_sample.tar.gz
Saving weekends_and_holidays.csv to weekends_and_holidays.csv
Saving times.tar.gz to times.tar (1).gz


In [3]:
import os

os.makedirs("data/ip_sample", exist_ok=True)
os.makedirs("data/times", exist_ok=True)


In [4]:
import tarfile

# Décompresser les séries IP
with tarfile.open("ip_addresses_sample.tar.gz") as tar:
    tar.extractall("data/ip_sample")

# Décompresser les timestamps
with tarfile.open("times.tar.gz") as tar:
    tar.extractall("data/times")


/tmp/ipython-input-697801761.py:5: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall("data/ip_sample")
/tmp/ipython-input-697801761.py:9: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall("data/times")


In [25]:
import glob

ip_files = glob.glob("data/ip_sample/ip_addresses_sample/agg_10_minutes/*.csv")
times_file = "data/times/times/times_10_minutes.csv"

print("Nb fichiers IP trouvés :", len(ip_files))
print("Exemple :", ip_files[:3])
print("Timestamp file exists :", os.path.exists(times_file))


Nb fichiers IP trouvés : 1000
Exemple : ['data/ip_sample/ip_addresses_sample/agg_10_minutes/100610.csv', 'data/ip_sample/ip_addresses_sample/agg_10_minutes/101.csv', 'data/ip_sample/ip_addresses_sample/agg_10_minutes/10125.csv']
Timestamp file exists : True


In [26]:
import pandas as pd

df_test = pd.read_csv(ip_files[0])
print(df_test.head())
print(df_test.columns)


   id_time  n_flows  n_packets  n_bytes  n_dest_asn  n_dest_ports  n_dest_ip  \
0        0       14         96    11462        12.0          13.0       12.0   
1        1       14         53     6413        11.0          12.0       12.0   
2        2       15         96    12698        11.0          12.0       12.0   
3        3        9         59     7822         7.0           9.0        9.0   
4        4       16         75     9862         9.0          14.0       13.0   

   tcp_udp_ratio_packets  tcp_udp_ratio_bytes  dir_ratio_packets  \
0                   1.00                 1.00               0.51   
1                   0.87                 0.87               0.51   
2                   0.91                 0.94               0.44   
3                   0.90                 0.93               0.59   
4                   0.99                 0.99               0.63   

   dir_ratio_bytes  avg_duration  avg_ttl  
0             0.54          3.84   177.36  
1             0.55    

# Bloc 1 — Clustering (k-means, k=3) pour définir 3 profils de trafic (6G)

Objectif : à partir des métriques CESNET (trafic agrégé 10 minutes), construire un dataset *catégorisé* en 3 groupes (clusters) interprétables comme :

- **eMBB+** (gros débit / gros volumes)
- **URLLC+** (communications critiques / interactif — proxy via patterns)
- **mMTC+** (beaucoup de flows/paquets, petits volumes)

Ce notebook ne fait **que** le Bloc 1 (clustering). Les blocs 2 (forecast) et 3 (allocation) viendront ensuite.

Pré-requis : tu as déjà dans ton répertoire de travail :
- `data/ip_sample/ip_addresses_sample/agg_10_minutes/*.csv`
- `data/times/times/times_10_minutes.csv`


In [7]:
import os, glob, random
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

SEED = 42
random.seed(SEED)
np.random.seed(SEED)


## 1) Chemins + paramètres

On va construire un dataset de points (IP, temps). Pour éviter de charger 40 Go :
- on limite à `MAX_IPS`
- on peut sous-échantillonner des timestamps par IP (`MAX_ROWS_PER_IP`)

Tu peux augmenter progressivement quand tout tourne.


In [8]:
IP_DIR = "data/ip_sample/ip_addresses_sample/agg_10_minutes"
TIMES_PATH = "data/times/times/times_10_minutes.csv"

MAX_IPS = 200          # commence à 200, puis 500, puis 1000
MAX_ROWS_PER_IP = 800  # sous-échantillonnage par IP (None = pas de limite)

FEATURES = [
    "n_bytes",
    "n_packets",
    "n_flows",
    "tcp_udp_ratio_bytes",
    "dir_ratio_bytes",
]

print("IP_DIR exists:", os.path.isdir(IP_DIR))
print("TIMES_PATH exists:", os.path.isfile(TIMES_PATH))


IP_DIR exists: True
TIMES_PATH exists: True


## 2) Charger les timestamps (10 minutes)

On ne garde que `id_time` et `timestamp`.


In [9]:
times = pd.read_csv(TIMES_PATH)[["id_time", "time"]].rename(columns={"time":"timestamp"})
times["timestamp"] = pd.to_datetime(times["timestamp"], errors="coerce")
times = times.dropna(subset=["timestamp"])
print(times.head())
print("nb timestamps:", len(times))


   id_time                  timestamp
0        0  2023-10-09 02:03:49+02:00
1        1  2023-10-09 02:13:49+02:00
2        2  2023-10-09 02:23:49+02:00
3        3  2023-10-09 02:33:49+02:00
4        4  2023-10-09 02:43:49+02:00
nb timestamps: 40298


/tmp/ipython-input-65587561.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  times["timestamp"] = pd.to_datetime(times["timestamp"], errors="coerce")


## 3) Lister les IPs disponibles + échantillonner

Chaque fichier CSV correspond à une entité (IP / point d'agrégation) et contient des métriques par pas de 10 minutes.


In [10]:
all_files = sorted(glob.glob(os.path.join(IP_DIR, "*.csv")))
print("nb fichiers trouvés:", len(all_files))

random.shuffle(all_files)
files = all_files[:MAX_IPS]
print("nb fichiers utilisés:", len(files))
print("exemple:", files[0] if files else None)


nb fichiers trouvés: 1000
nb fichiers utilisés: 200
exemple: data/ip_sample/ip_addresses_sample/agg_10_minutes/556848.csv


## 4) Construire la table (IP, temps) → features

On charge chaque IP, on merge les timestamps, on garde les features, puis on concatène.

Astuce : on extrait un identifiant IP depuis le nom de fichier (ex: `355332.csv` → ip_id=355332).


In [11]:
def load_one_ip(path: str) -> pd.DataFrame:
    ip_id = os.path.splitext(os.path.basename(path))[0]
    df = pd.read_csv(path)
    # Merge temps
    df = df.merge(times, on="id_time", how="inner")

    # Garder colonnes utiles
    keep = ["id_time", "timestamp"] + FEATURES
    df = df[keep].copy()
    df["ip_id"] = ip_id

    # Nettoyage: remplacer inf / NaN
    df = df.replace([np.inf, -np.inf], np.nan).dropna()

    # Sous-échantillonner si besoin
    if MAX_ROWS_PER_IP is not None and len(df) > MAX_ROWS_PER_IP:
        df = df.sample(MAX_ROWS_PER_IP, random_state=SEED)

    return df

parts = []
for i, p in enumerate(files, 1):
    if i % 50 == 0:
        print(f"... {i}/{len(files)}")
    try:
        parts.append(load_one_ip(p))
    except Exception as e:
        # On skip sans casser le notebook
        print("skip", p, "->", str(e)[:120])

df_points = pd.concat(parts, ignore_index=True)
print(df_points.head())
print("shape:", df_points.shape)
print("nb ip_id:", df_points["ip_id"].nunique())


/tmp/ipython-input-3205789934.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([np.inf, -np.inf], np.nan).dropna()


... 50/200
... 100/200
... 150/200
... 200/200
   id_time                  timestamp  n_bytes  n_packets  n_flows  \
0    22761  2024-03-15 03:40:52+01:00      480          5        3   
1    23744  2024-03-21 23:30:52+01:00    42685        165        7   
2     3288  2023-10-31 22:03:49+01:00       92          2        2   
3    21254  2024-03-04 16:30:52+01:00    12422         30        4   
4    20660  2024-02-29 13:30:52+01:00      956          6        5   

   tcp_udp_ratio_bytes  dir_ratio_bytes   ip_id  
0                 1.00             0.38  556848  
1                 0.09             0.62  556848  
2                 1.00             0.43  556848  
3                 1.00             0.28  556848  
4                 0.31             0.19  556848  
shape: (146390, 8)
nb ip_id: 200


## 5) Pré-traitement : log + standardisation

Les variables `n_bytes`, `n_packets`, `n_flows` ont souvent des ordres de grandeur énormes.
On applique `log1p` sur ces trois-là, puis on standardise toutes les features.


In [12]:
df = df_points.copy()

for col in ["n_bytes", "n_packets", "n_flows"]:
    df[col] = np.log1p(df[col].astype(float))

X = df[FEATURES].astype(float).values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("X_scaled:", X_scaled.shape)


X_scaled: (146390, 5)


## 6) K-Means (k=3)

On entraîne un k-means et on calcule un score de silhouette (proxy de qualité de clustering).


In [13]:
k = 3
kmeans = KMeans(n_clusters=k, random_state=SEED, n_init="auto")
clusters = kmeans.fit_predict(X_scaled)

df["cluster"] = clusters

# Silhouette: attention, coûteux si énorme. Ici, ça passe avec échantillonnage.
# On sample au besoin.
SAMPLE_FOR_SIL = min(20000, len(df))
idx = np.random.choice(len(df), size=SAMPLE_FOR_SIL, replace=False)
score = silhouette_score(X_scaled[idx], clusters[idx])

print("silhouette (approx):", score)
print(df["cluster"].value_counts())


silhouette (approx): 0.2585522127131743
cluster
1    77496
0    53720
2    15174
Name: count, dtype: int64


## 7) Interpréter les clusters

On regarde les stats par cluster (moyennes) pour interpréter :
- cluster avec `n_bytes` élevé → eMBB+
- cluster avec beaucoup de petits paquets/flows et ratios caractéristiques → mMTC+
- cluster restant → URLLC+ (interprétation via proxies disponibles)

Note : la latence n'est pas directement observable ici. URLLC est donc une **interprétation** basée sur des proxies (packets/flows/ratios) et sur le fait que ce cluster n'est ni "gros débit" ni "massive IoT".


In [14]:
summary = df.groupby("cluster")[FEATURES].agg(["mean","median","std"]).round(3)
summary


n_bytes                n_packets               n_flows                \
           mean  median    std      mean median    std    mean median    std   
cluster                                                                        
0         9.703   9.785  1.452     4.315  4.234  0.917   2.176  2.079  0.563   
1         6.140   6.004  1.136     2.139  2.079  0.665   1.449  1.386  0.418   
2        13.942  13.570  2.012     7.596  7.129  1.636   3.432  3.135  1.279   

        tcp_udp_ratio_bytes               dir_ratio_bytes                
                       mean median    std            mean median    std  
cluster                                                                  
0                     0.828    1.0  0.316           0.388   0.35  0.258  
1                     0.875    1.0  0.264           0.505   0.45  0.281  
2                     0.826    1.0  0.321           0.248   0.11  0.290

## 8) Sauvegarder un dataset labellisé (points)

On sauvegarde (ip_id, timestamp, cluster, features) pour les blocs suivants.


In [15]:
OUT_PATH = "cesnet_points_clustered.csv"
df_out = df[["ip_id","id_time","timestamp"] + FEATURES + ["cluster"]].copy()
df_out.to_csv(OUT_PATH, index=False)
print("saved:", OUT_PATH, "rows:", len(df_out))


saved: cesnet_points_clustered.csv rows: 146390


## 9) (Optionnel) Construire la **demande par cluster** au cours du temps

Pour préparer le Bloc 2 (prévision), tu voudras des séries temporelles :

- `D_cluster0(t)` = somme des bytes des points classés cluster 0 au temps t
- idem cluster 1, cluster 2

Ici on te montre un exemple global (toutes IPs confondues). Ensuite, on pourra faire la même chose **par IP**.


In [16]:
# ATTENTION: ici on utilise n_bytes log-transformé (df["n_bytes"]).
# Pour une demande en bytes, préfère partir de df_points brut.

# On repart du df_points brut et on joint les clusters (par index de ligne)
df_global = df_points.copy()
df_global["cluster"] = df["cluster"].values

# Demande = somme n_bytes (brut) par timestamp et cluster
agg = (df_global
       .groupby(["timestamp","cluster"])["n_bytes"]
       .sum()
       .reset_index())

pivot = agg.pivot(index="timestamp", columns="cluster", values="n_bytes").fillna(0).sort_index()
pivot.head(), pivot.shape


(cluster                          0       1          2
 timestamp                                            
 2023-10-09 02:03:49+02:00      0.0  2177.0  9986407.0
 2023-10-09 02:13:49+02:00      0.0   456.0        0.0
 2023-10-09 02:33:49+02:00      0.0   480.0        0.0
 2023-10-09 02:43:49+02:00  24880.0     0.0        0.0
 2023-10-09 02:53:49+02:00   7776.0     0.0        0.0,
 (38155, 3))

In [27]:
import joblib

joblib.dump(kmeans, "kmeans_3clusters.pkl")
joblib.dump(scaler, "scaler.pkl")

print("Sauvegardé dans Drive :")
print("- kmeans_3clusters.pkl")
print("- scaler.pkl")


Sauvegardé dans Drive :
- kmeans_3clusters.pkl
- scaler.pkl


In [28]:
import os
print(os.listdir("."))



['data', 'kmeans_3clusters.pkl', 'scaler.pkl']
